In [7]:
import pandas as pd
import os
from pathlib import Path
from collections import Counter

In [2]:
etd_dir = '/cmnfs/data/proteomics/ProteomeTools/ETD/parquet'
data_frames = list()
for file_name in os.listdir(etd_dir):
    data_frames.append(pd.read_parquet(Path(etd_dir) / file_name))

In [3]:
for df in data_frames:
    print(len(df))

2588
2083
2365
2554
2257
2885
2569
2351
2287
2749


In [50]:
all_seqs = pd.concat(data_frames, axis=0)

In [51]:
all_seqs.reset_index(inplace=True)
all_seqs.drop('index', inplace=True, axis=1)

In [52]:
def count_non_zero(intensities):
    intensities = intensities[intensities != 0]
    intensities = intensities[intensities != -1]
    return len(intensities)

In [53]:
all_seqs.loc[:, 'intensities_raw'].apply(count_non_zero).describe()

count    24688.000000
mean         3.915506
std          2.415584
min          0.000000
25%          2.000000
50%          3.000000
75%          5.000000
max         16.000000
Name: intensities_raw, dtype: float64

In [58]:
total_seqs = len(all_seqs)
train_df = all_seqs.sample(frac=0.7)
all_seqs.drop(train_df.index, axis=0, inplace=True)
val_df = all_seqs.sample(frac=2/3)
test_df = all_seqs.drop(val_df.index, axis=0)
print(f'Train set has {len(train_df) / total_seqs:.2%} of all sequences')
print(f'Validation set has {len(val_df) / total_seqs:.2%} of all sequences')
print(f'Test set has {len(test_df) / total_seqs:.2%} of all sequences')

Train set has 70.00% of all sequences
Validation set has 20.00% of all sequences
Test set has 10.00% of all sequences


In [61]:
train_df.to_parquet(Path(etd_dir) / 'etd_data_train.parquet')
val_df.to_parquet(Path(etd_dir) / 'etd_data_val.parquet')
test_df.to_parquet(Path(etd_dir) / 'etd_data_test.parquet')